# Extract Spike Times

Code to extract spike times from cluster files and create the 'times_' files. 

In [ ]:
# ...
from hsntools.paths import Paths

In [ ]:
# Add local folder with `sort` module
import sys
sys.path.append('..')
from sort.io import ...

## Paths & Files

In [ ]:
# Set the project path
project_path = '/path/'

In [ ]:
# Define subject information
subj = 'subjX'
session = 'sessionX'

In [ ]:
# Initialize paths object
paths = Paths(project_path, subj, session)

In [ ]:
# Check the paths for the sorting folder
paths.sorting

In [ ]:
# Check the paths for the spikes folder
paths.spikes

## Settings

In [ ]:
# Define spike sorting related settings
polarity = 'neg'       # 'neg' / 'pos'
user = 'xxx'           # 3 character user tag

## Extract Example Channel

The following code can be used to extract units from an example channel. 

The outputs of this process are a hdf5 file for each unit (so there can be multiple output files per channel).

This process assumes the combinato organization of data, including that the folder of interest is named by the channel label.

In [ ]:
# Set example channel to run
channel = 'XX'

In [ ]:
# Extract the units from a single specified channel
process_combinato_data(channel, paths.sorting, polarity, user, paths.spikes)

## Extract All Channels

The following example code can used to run the extraction process across all available spike sorted channels in a specified folder.

Note that this code mimics what is available in the `extract_sorted` script.

In [ ]:
# Extract units from all channels included in the sorting folder
for chandir in paths.get_subfolders('sorting'):
    print('Processing: ', chandir)
    process_combinato_data(chandir, paths.sorting, polarity, user, paths.spikes)